In [26]:
import numpy as np
import pandas as pd
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import MEstimateEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Load Dataset
df = pd.read_csv('./data/train_data.csv')

# Feature Engineering
class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['IsActive_by_CreditCard'] = X_copy['HasCrCard'] * X_copy['IsActiveMember']
        X_copy['Products_Per_Tenure'] = X_copy['Tenure'] / (X_copy['NumOfProducts'] + 1)
        X_copy['ZeroBalance'] = (X_copy['Balance'] == 0).astype(np.uint8)
        X_copy['AgeCat'] = np.round(X_copy.Age / 20).astype(np.uint16)
        return X_copy

# Text Vectorization
class Vectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=1000, cols=['Surname'], n_components=3):
        self.max_features = max_features
        self.cols = cols
        self.n_components = n_components
    
    def fit(self, X, y=None):
        self.vectorizer_dict = {}
        self.decomposer_dict = {}
        for col in self.cols:
            self.vectorizer_dict[col] = TfidfVectorizer(max_features=self.max_features).fit(X[col].astype(str))
            self.decomposer_dict[col] = TruncatedSVD(n_components=self.n_components).fit(
                self.vectorizer_dict[col].transform(X[col].astype(str))
            )
        return self
    
    def transform(self, X):
        vectorized_df = pd.DataFrame()
        for col in self.cols:
            vectorized = self.vectorizer_dict[col].transform(X[col].astype(str))
            vectorized = self.decomposer_dict[col].transform(vectorized)
            vectorized_df = pd.concat([vectorized_df, pd.DataFrame(vectorized)], axis=1)
        return pd.concat([X.reset_index(drop=True), vectorized_df], axis=1)

# Preprocessing
def preprocess_data(df):
    df = df.copy()
    categorical_features = ['Geography', 'Gender']
    encoder = MEstimateEncoder()
    df[categorical_features] = encoder.fit_transform(df[categorical_features], df['Exited'])
    df = FeatureEngineering().transform(df)
    df = Vectorizer(cols=['Surname']).fit_transform(df)
    features = ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
                'NumOfProducts', 'IsActiveMember', 'Products_Per_Tenure', 'ZeroBalance', 'AgeCat']
    return df[features + ['Exited']]

df = preprocess_data(df)

# Train/Test Split
X = df.drop(columns=['Exited'])
y = df['Exited']
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter Tuning with Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0),
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        scores.append(f1_score(y_val, y_pred))
    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best Model Selection
best_params = study.best_params
lgb_model = LGBMClassifier(**best_params, random_state=42)
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
xgb_model = XGBClassifier(n_estimators=200, random_state=42)
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Voting Ensemble
voting_clf = VotingClassifier(
    estimators=[('lgb', lgb_model), ('rf', rf_model), ('xgb', xgb_model), ('lr', log_reg)],
    voting='soft'
)

# Cross-validation Evaluation
scores = []
for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    voting_clf.fit(X_train, y_train)
    y_pred = voting_clf.predict(X_val)
    scores.append(f1_score(y_val, y_pred))

# Print Final Results
print(f"\nFinal Cross-Validation F1 Score: {np.mean(scores):.4f} ± {np.std(scores):.4f}")


[I 2025-02-05 15:47:47,513] A new study created in memory with name: no-name-501920b6-edb3-4888-85bf-9da3a7a41b41


[LightGBM] [Info] Number of positive: 24360, number of negative: 90503
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000647 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 114863, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212079 -> initscore=-1.312441
[LightGBM] [Info] Start training from score -1.312441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2025-02-05 15:47:48,151] Trial 0 finished with value: 0.03835775260298209 and parameters: {'max_depth': 3, 'learning_rate': 0.007651728706999581, 'n_estimators': 74, 'subsample': 0.8774732806872161, 'colsample_bytree': 0.7232830719720642, 'reg_alpha': 8.582775022554566, 'reg_lambda': 3.0699255053358376}. Best is trial 0 with value: 0.03835775260298209.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-02-05 15:47:48,935] Trial 1 finished with value: 0.0 and parameters: {'max_depth': 3, 'learning_rate': 0.0018369249250912022, 'n_estimators': 110, 'subsample': 0.7278359631010775, 'colsample_bytree': 0.5737357987508334, 'reg_alpha': 4.23180235770547, 'reg_lambda': 7.236834952696313}. Best is trial 0 with value: 0.03835775260298209.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 24360, number of negative: 90503
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000587 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 114863, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212079 -> initscore=-1.312441
[LightGBM] [Info] Start training from score -1.312441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2025-02-05 15:49:11,670] Trial 48 finished with value: 0.625127868104291 and parameters: {'max_depth': 3, 'learning_rate': 0.044722590622458286, 'n_estimators': 197, 'subsample': 0.7279624668220234, 'colsample_bytree': 0.9229075708955905, 'reg_alpha': 0.849406174474999, 'reg_lambda': 7.4495321008947935}. Best is trial 32 with value: 0.6391789878657874.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-02-05 15:49:13,794] Trial 49 finished with value: 0.6358931893865105 and parameters: {'max_depth': 7, 'learning_rate': 0.029668077945439377, 'n_estimators': 210, 'subsample': 0.7363897176467731, 'colsample_bytree': 0.8966136003679921, 'reg_alpha': 1.644163630176453, 'reg_lambda': 6.628584071659142}. Best is trial 32 with value: 0.6391789878657874.


AttributeError: 'super' object has no attribute '__sklearn_tags__'